In [4]:
from datetime import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [5]:
def fundamentals_from_yahoo(ticker):
    url="https://finance.yahoo.com/quote/%s/financials" % ticker
    soup = BeautifulSoup(urlopen(url))
    rows = soup.find_all(text='Gross Profit')[0].parent.parent.parent.parent.contents
    
    fundis=[]
    for row in rows:
        if len(row.contents) > 3:
            try:
                key = row.contents[0].contents[0].text
                if key == 'Revenue':
                    fundis.append(["Date"] + 
                                  [datetime.strptime(col.contents[0].text, '%m/%d/%Y').date()
                                   for col in row.contents[1:]])
                else:
                    fundis.append([key] + [
                        int(col.contents[0].text.replace(',',''))/1e6 
                        for col in row.contents[1:]])
            except:
                pass

    fundis = np.transpose(fundis)
    df = pd.DataFrame(data=fundis[1:, 1:], columns=fundis[0, 1:], index=fundis[1:, 0])
    #df.index.name = "Date"
    return df

In [6]:
fundis = fundamentals_from_yahoo('AAPL')
fundis

,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Total Operating Expenses,Operating Income or Loss,Total Other Income/Expenses Net,Earnings Before Interest and Taxes,Interest Expense,Income Before Tax,Income Tax Expense,Net Income From Continuing Ops,Net Income,Net Income Applicable To Common Shares
2018-09-29,265.595,163.756,101.839,14.236,16.705,194.697,70.898,2.005,70.898,-3.24,72.903,13.372,59.531,59.531,59.531
2017-09-30,229.234,141.048,88.186,11.581,15.261,167.89,61.344,2.745,61.344,-2.323,64.089,15.738,48.351,48.351,48.351
2016-09-24,215.639,131.376,84.263,10.045,14.194,155.615,60.024,1.348,60.024,-1.456,61.372,15.685,45.687,45.687,45.687
2015-09-26,233.715,140.089,93.626,8.067,14.329,162.485,71.23,1.285,71.23,-0.733,72.515,19.121,53.394,53.394,53.394
